# exp08: AirREGI 翌日入電予測（Holdout Validation）

**exp03との違い**: 検証手法をHoldout法に変更

- ベースライン: exp03
- 特徴量: exp03と同じ（35特徴量）
- モデル: exp03と同じ
- **変更点**: 訓練データの最後の60日をValidationセットとして固定分割（Holdout法）

## Holdout法とは
- 訓練データを「Train」と「Validation」に1回だけ分割
- Train: モデル学習に使用
- Validation: ハイパーパラメータ調整・早期停止に使用
- Test: 最終評価（変更なし）

## 時系列データでのHoldout
```
├─────────────────── Train ──────────────────┤── Val ──┤── Test ──┤
2018-06                              2019-11  2019-12   2020-01
```

## 重要な原則（exp03から継承）
1. 未来のデータを使わない（ラグ特徴量は過去のデータのみ）
2. 移動平均も過去のデータのみを使用
3. 訓練データとテストデータは時系列で分割

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ==================================================================================
# Step 1: データの読み込みと前処理
# ==================================================================================

def load_and_preprocess_data():
    """
    各データセットを読み込み、日付型に変換
    """
    print("=" * 80)
    print("Step 1: データの読み込み")
    print("=" * 80)
    
    # データ読み込み
    calender = pd.read_csv('../input/calender_data.csv')
    cm_data = pd.read_csv('../input/cm_data.csv')
    gt_service = pd.read_csv('../input/gt_service_name.csv')
    acc_get = pd.read_csv('../input/regi_acc_get_data_transform.csv')
    call_data = pd.read_csv('../input/regi_call_data_transform.csv')
    
    # 日付カラムをdatetime型に変換
    calender['cdr_date'] = pd.to_datetime(calender['cdr_date'])
    cm_data['cdr_date'] = pd.to_datetime(cm_data['cdr_date'])
    acc_get['cdr_date'] = pd.to_datetime(acc_get['cdr_date'])
    call_data['cdr_date'] = pd.to_datetime(call_data['cdr_date'])
    gt_service['week'] = pd.to_datetime(gt_service['week'])
    
    print(f"\nカレンダーデータ: {calender.shape}")
    print(f"CMデータ: {cm_data.shape}")
    print(f"Google Trendsデータ: {gt_service.shape}")
    print(f"アカウント取得データ: {acc_get.shape}")
    print(f"入電データ（目的変数）: {call_data.shape}")
    
    return calender, cm_data, gt_service, acc_get, call_data

In [3]:
# ==================================================================================
# Step 2: データの統合
# ==================================================================================

def merge_datasets(calender, cm_data, gt_service, acc_get, call_data):
    """
    全てのデータセットを統合
    Google Trendsは週次データなので日次に展開
    """
    print("\n" + "=" * 80)
    print("Step 2: データの統合")
    print("=" * 80)
    
    # メインデータ（入電数）を基準にマージ
    df = call_data.copy()
    print(f"\nベースデータ: {df.shape}")
    
    # カレンダー情報をマージ
    df = df.merge(calender, on='cdr_date', how='left')
    print(f"カレンダー情報マージ後: {df.shape}")
    
    # CM情報をマージ
    df = df.merge(cm_data, on='cdr_date', how='left')
    print(f"CM情報マージ後: {df.shape}")
    
    # アカウント取得数をマージ
    df = df.merge(acc_get, on='cdr_date', how='left')
    print(f"アカウント取得数マージ後: {df.shape}")
    
    # Google Trendsデータは週次なので日次に展開
    print("\nGoogle Trendsデータを週次→日次に展開...")
    gt_service_daily = []
    for idx, row in gt_service.iterrows():
        week_start = row['week']
        for i in range(7):
            date = week_start + timedelta(days=i)
            gt_service_daily.append({
                'cdr_date': date, 
                'search_cnt': row['search_cnt']
            })
    
    gt_daily = pd.DataFrame(gt_service_daily)
    df = df.merge(gt_daily, on='cdr_date', how='left')
    print(f"Google Trendsマージ後: {df.shape}")
    
    # 欠損値の確認
    print("\n欠損値の数:")
    print(df.isnull().sum().sort_values(ascending=False).head(10))
    
    return df

In [4]:
# ==================================================================================
# Step 3: 基本的な時系列特徴量の作成
# ==================================================================================

def create_basic_time_features(df):
    """
    日付から派生する基本的な時系列特徴量を作成
    これらは未来の情報を使わないので安全
    """
    print("\n" + "=" * 80)
    print("Step 3: 基本的な時系列特徴量の作成")
    print("=" * 80)
    
    df = df.copy()
    
    # 年月日の特徴量
    df['year'] = df['cdr_date'].dt.year
    df['month'] = df['cdr_date'].dt.month
    df['day_of_month'] = df['cdr_date'].dt.day
    df['quarter'] = df['cdr_date'].dt.quarter
    df['day_of_year'] = df['cdr_date'].dt.dayofyear
    
    # 週の情報（既にwoy, womがあるが念のため）
    df['week_of_year'] = df['cdr_date'].dt.isocalendar().week
    
    # 経過日数（データの開始日からの日数）
    df['days_from_start'] = (df['cdr_date'] - df['cdr_date'].min()).dt.days
    
    # 曜日情報（既にdowがあるが確認）
    # dow: 1=月, 2=火, ..., 7=日
    
    # 月初・月末フラグ
    df['is_month_start'] = (df['day_of_month'] <= 5).astype(int)
    df['is_month_end'] = (df['day_of_month'] >= 25).astype(int)
    
    print("\n作成した基本特徴量:")
    time_features = ['year', 'month', 'day_of_month', 'quarter', 'day_of_year', 
                     'week_of_year', 'days_from_start', 'is_month_start', 'is_month_end']
    print(time_features)
    
    return df

In [5]:
# ==================================================================================
# Step 4: ラグ特徴量の作成（データリーケージに注意！）
# ==================================================================================

def create_lag_features(df, target_col='call_num', lags=[1, 2, 3, 5, 7, 14, 30]):
    """
    ラグ特徴量（過去のデータ）を作成
    
    重要: 
    - shift()を使って、未来の情報が混入しないようにする
    - lag=1は1日前のデータ、lag=7は7日前のデータ
    
    例: 
    日付        call_num    lag_1    lag_7
    2018-06-01    183        NaN      NaN
    2018-06-02      0        183      NaN
    2018-06-08     96         0       183  <- 7日前のデータ
    """
    print("\n" + "=" * 80)
    print("Step 4: ラグ特徴量の作成（データリーケージ防止）")
    print("=" * 80)
    
    df = df.copy()
    
    print(f"\n目的変数: {target_col}")
    print(f"作成するラグ: {lags}")
    
    for lag in lags:
        col_name = f'lag_{lag}'
        df[col_name] = df[target_col].shift(lag)
        print(f"  作成: {col_name} (shift={lag})")
    
    # 最初のN日はラグ特徴量がNaNになる
    print(f"\n注意: 最初の{max(lags)}日間はラグ特徴量がNaNになります")
    print(f"例: lag_30は最初の30日間がNaN")
    
    return df

In [6]:
# ==================================================================================
# Step 5: 移動平均特徴量の作成（データリーケージに注意！）
# ==================================================================================

def create_rolling_features(df, target_col='call_num', windows=[3, 7, 14, 30]):
    """
    移動平均・移動標準偏差を作成
    
    重要:
    - rolling().mean() を使う前に shift(1) を適用
    - これにより、当日のデータが移動平均に含まれない（リーケージ防止）
    
    例（window=3の場合）:
    日付        call_num    過去3日の平均
    2018-06-01    183        NaN
    2018-06-02      0        NaN
    2018-06-03      0        NaN
    2018-06-04    213        61.0  <- (183+0+0)/3 = 61.0
    """
    print("\n" + "=" * 80)
    print("Step 5: 移動平均特徴量の作成（データリーケージ防止）")
    print("=" * 80)
    
    df = df.copy()
    
    print(f"\n目的変数: {target_col}")
    print(f"移動平均ウィンドウ: {windows}")
    
    for window in windows:
        # 移動平均（当日を含まない＝shift(1)してからrolling）
        ma_col = f'ma_{window}'
        df[ma_col] = df[target_col].shift(1).rolling(window=window, min_periods=1).mean()
        print(f"  作成: {ma_col} (過去{window}日間の平均)")
        
        # 移動標準偏差（変動の大きさを捉える）
        std_col = f'ma_std_{window}'
        df[std_col] = df[target_col].shift(1).rolling(window=window, min_periods=1).std()
        print(f"  作成: {std_col} (過去{window}日間の標準偏差)")
    
    print("\nなぜshift(1)が必要か:")
    print("  × 悪い例: rolling(3).mean() → 当日含む3日間の平均（リーケージ！）")
    print("  ○ 良い例: shift(1).rolling(3).mean() → 過去3日間の平均（安全）")
    
    return df

In [7]:
# ==================================================================================
# Step 6: その他の集約特徴量
# ==================================================================================

def create_aggregated_features(df):
    """
    その他の有用な集約特徴量を作成
    """
    print("\n" + "=" * 80)
    print("Step 6: その他の集約特徴量")
    print("=" * 80)
    
    df = df.copy()
    
    # CM効果の累積（過去7日間のCM実施回数）
    # これも当日を含まないようにshift(1)
    df['cm_7d'] = df['cm_flg'].shift(1).rolling(window=7, min_periods=1).sum()
    print("  作成: cm_7d (過去7日間のCM実施回数)")
    
    # Google Trendsの移動平均（ノイズ除去）
    df['gt_ma_7'] = df['search_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
    print("  作成: gt_ma_7 (過去7日間のGoogle Trends平均)")
    
    # アカウント取得数の移動平均
    df['acc_ma_7'] = df['acc_get_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
    print("  作成: acc_ma_7 (過去7日間のアカウント取得平均)")
    
    # 曜日ごとの過去平均（同じ曜日の過去データの平均）
    # これは少し高度だが、曜日パターンを捉えるのに有効
    print("\n  作成中: dow_avg (同じ曜日の過去平均)...")
    df['dow_avg'] = np.nan
    for dow in df['dow'].unique():
        mask = df['dow'] == dow
        # 各行について、その行より前のデータの平均を計算
        df.loc[mask, 'dow_avg'] = df.loc[mask, 'call_num'].shift(1).expanding().mean()
    
    return df

In [8]:
# ==================================================================================
# Step 7: 特徴量の選択とデータ分割（翌日予測に変更）
# ==================================================================================

def select_features_and_split(df, test_months=3):
    """
    特徴量を選択し、訓練データとテストデータに分割
    
    重要: 時系列データは時間順に分割する
    **exp03変更点**: 目的変数を call_num.shift(-1) に変更（翌日予測）
    """
    print("\n" + "=" * 80)
    print("Step 7: 特徴量選択とデータ分割")
    print("=" * 80)
    
    # 🚨 exp03変更点: 翌日の入電数を目的変数にする
    df = df.copy()
    df['target_next_day'] = df['call_num'].shift(-1)
    print("\n🎯 exp03: 目的変数を翌日の入電数に設定 (call_num.shift(-1))")
    
    # 最後の行はtargetがNaNになるので削除
    df = df.dropna(subset=['target_next_day']).reset_index(drop=True)
    print(f"翌日データがない最終行を削除: {len(df)}行")
    
    # 平日のみを使用（土日は入電数が0なので予測不要）
    df_model = df[df['dow'].isin([1, 2, 3, 4, 5])].copy().reset_index(drop=True)
    print(f"\n平日のみに絞り込み: {len(df)} → {len(df_model)}行")
    
    # 特徴量リスト（exp01と同じ）
    feature_cols = [
        # 基本的な時系列特徴量
        'dow', 'day_of_month', 'month', 'quarter', 'year', 
        'days_from_start', 'day_of_year', 'week_of_year',
        'is_month_start', 'is_month_end',
        
        # カレンダー特徴量
        'woy', 'wom', 'day_before_holiday_flag',
        
        # 外部データ
        'cm_flg', 'acc_get_cnt', 'search_cnt',
        
        # 集約特徴量
        'cm_7d', 'gt_ma_7', 'acc_ma_7', 'dow_avg',
        
        # ラグ特徴量
        'lag_1', 'lag_2', 'lag_3', 'lag_5', 'lag_7', 'lag_14', 'lag_30',
        
        # 移動平均特徴量
        'ma_3', 'ma_7', 'ma_14', 'ma_30',
        'ma_std_3', 'ma_std_7', 'ma_std_14', 'ma_std_30'
    ]
    
    print(f"\n使用する特徴量数: {len(feature_cols)}")
    print("\n特徴量のカテゴリ:")
    print(f"  - 基本時系列: 10個")
    print(f"  - カレンダー: 3個")
    print(f"  - 外部データ: 3個")
    print(f"  - 集約特徴量: 4個")
    print(f"  - ラグ特徴量: 7個")
    print(f"  - 移動平均: 8個")
    
    # 時系列分割（最後の3ヶ月をテストデータ）
    split_date = df_model['cdr_date'].max() - pd.Timedelta(days=30*test_months)
    
    train_df = df_model[df_model['cdr_date'] < split_date].copy()
    test_df = df_model[df_model['cdr_date'] >= split_date].copy()
    
    print(f"\n時系列分割:")
    print(f"  訓練データ期間: {train_df['cdr_date'].min()} ~ {train_df['cdr_date'].max()}")
    print(f"  テストデータ期間: {test_df['cdr_date'].min()} ~ {test_df['cdr_date'].max()}")
    print(f"  訓練データ数: {len(train_df)}行")
    print(f"  テストデータ数: {len(test_df)}行")
    
    # 欠損値がある行を削除（ラグ特徴量の初期値等）
    train_clean = train_df.dropna(subset=feature_cols + ['target_next_day'])
    test_clean = test_df.dropna(subset=feature_cols + ['target_next_day'])
    
    print(f"\n欠損値除去後:")
    print(f"  訓練データ数: {len(train_clean)}行")
    print(f"  テストデータ数: {len(test_clean)}行")
    
    # X（特徴量）とy（目的変数）に分割
    X_train = train_clean[feature_cols]
    y_train = train_clean['target_next_day']  # 🚨 exp03: 翌日の値
    X_test = test_clean[feature_cols]
    y_test = test_clean['target_next_day']    # 🚨 exp03: 翌日の値
    
    # メタ情報も保存（日付など）
    train_meta = train_clean[['cdr_date', 'call_num', 'target_next_day']]
    test_meta = test_clean[['cdr_date', 'call_num', 'target_next_day']]
    
    return X_train, X_test, y_train, y_test, train_meta, test_meta, feature_cols

In [9]:
# ==================================================================================
# Step 8: 特徴量の重要度確認用関数
# ==================================================================================

def analyze_features(X_train, y_train, feature_cols):
    """
    特徴量の基本統計と相関を確認
    """
    print("\n" + "=" * 80)
    print("Step 8: 特徴量の分析")
    print("=" * 80)
    
    # 基本統計量
    print("\n特徴量の基本統計:")
    print(X_train.describe().T[['mean', 'std', 'min', 'max']].head(10))
    
    # 目的変数との相関
    print("\n目的変数（翌日入電数）との相関（上位10）:")
    correlations = pd.DataFrame({
        'feature': feature_cols,
        'correlation': [X_train[col].corr(y_train) for col in feature_cols]
    }).sort_values('correlation', ascending=False, key=abs)
    
    print(correlations.head(10))
    
    return correlations

In [10]:
# ==================================================================================
# メイン実行関数
# ==================================================================================

def main():
    """
    全ての処理を実行
    """
    print("\n")
    print("*" * 80)
    print("exp08: AirREGI ヘルプデスク 翌日入電予測 (Holdout Validation)")
    print("*" * 80)
    
    # Step 1: データ読み込み
    calender, cm_data, gt_service, acc_get, call_data = load_and_preprocess_data()
    
    # Step 2: データ統合
    df = merge_datasets(calender, cm_data, gt_service, acc_get, call_data)
    
    # Step 3: 基本時系列特徴量
    df = create_basic_time_features(df)
    
    # Step 4: ラグ特徴量
    df = create_lag_features(df, target_col='call_num', lags=[1, 2, 3, 5, 7, 14, 30])
    
    # Step 5: 移動平均特徴量
    df = create_rolling_features(df, target_col='call_num', windows=[3, 7, 14, 30])
    
    # Step 6: その他集約特徴量
    df = create_aggregated_features(df)
    
    # Step 7: 特徴量選択とデータ分割（翌日予測版）
    X_train, X_test, y_train, y_test, train_meta, test_meta, feature_cols = \
        select_features_and_split(df, test_months=3)
    
    # Step 8: 特徴量分析
    correlations = analyze_features(X_train, y_train, feature_cols)
    
    # データ保存
    print("\n" + "=" * 80)
    print("データの保存")
    print("=" * 80)
    
    # output ディレクトリが存在しない場合は作成
    import os
    output_dir = '../output/exp08'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"  {output_dir} ディレクトリを作成しました")
    
    train_data = pd.concat([train_meta.reset_index(drop=True), 
                           X_train.reset_index(drop=True)], axis=1)
    test_data = pd.concat([test_meta.reset_index(drop=True), 
                          X_test.reset_index(drop=True)], axis=1)
    
    train_data.to_csv(f'{output_dir}/train_features.csv', index=False)
    test_data.to_csv(f'{output_dir}/test_features.csv', index=False)
    correlations.to_csv(f'{output_dir}/feature_correlations.csv', index=False)
    
    print("\n保存完了:")
    print(f"  - {output_dir}/train_features.csv")
    print(f"  - {output_dir}/test_features.csv")
    print(f"  - {output_dir}/feature_correlations.csv")
    
    print("\n" + "=" * 80)
    print("特徴量エンジニアリング完了！")
    print("=" * 80)
    print(f"\n訓練データ: {X_train.shape}")
    print(f"テストデータ: {X_test.shape}")
    print(f"特徴量数: {len(feature_cols)}")
    print(f"\n🎯 目的変数: 翌日の入電数 (call_num.shift(-1))")
    
    return df, X_train, X_test, y_train, y_test, feature_cols

In [11]:
# ==================================================================================
# 実行
# ==================================================================================

if __name__ == "__main__":
    df, X_train, X_test, y_train, y_test, feature_cols = main()



********************************************************************************
exp08: AirREGI ヘルプデスク 翌日入電予測 (Holdout Validation)
********************************************************************************
Step 1: データの読み込み

カレンダーデータ: (670, 10)
CMデータ: (762, 2)
Google Trendsデータ: (109, 2)
アカウント取得データ: (701, 2)
入電データ（目的変数）: (670, 2)

Step 2: データの統合

ベースデータ: (670, 2)
カレンダー情報マージ後: (670, 11)
CM情報マージ後: (670, 12)
アカウント取得数マージ後: (670, 13)

Google Trendsデータを週次→日次に展開...
Google Trendsマージ後: (670, 14)

欠損値の数:
holiday_name               632
cdr_date                     0
call_num                     0
dow                          0
dow_name                     0
woy                          0
wom                          0
doy                          0
financial_year               0
day_before_holiday_flag      0
dtype: int64

Step 3: 基本的な時系列特徴量の作成

作成した基本特徴量:
['year', 'month', 'day_of_month', 'quarter', 'day_of_year', 'week_of_year', 'days_from_start', 'is_month_start', 'is_month_end']

Step 4

---

# モデル学習セクション（exp01と同じ）

In [12]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)

# =========================
# 0) Load（既に上で作成済み）
# =========================
print("Train:", X_train.shape, "Test:", X_test.shape)
print("Example features:", feature_cols[:10])

Train: (391, 35) Test: (65, 35)
Example features: ['dow', 'day_of_month', 'month', 'quarter', 'year', 'days_from_start', 'day_of_year', 'week_of_year', 'is_month_start', 'is_month_end']


In [13]:
# =========================
# 1) Metrics helper（WAPE追加版）
# =========================
def evaluate(y_true, y_pred):
    """
    評価指標を計算（MAE, RMSE, R2, WAPE）
    """
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import numpy as np
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    
    # WAPE (Weighted Absolute Percentage Error)
    # WAPE = sum(|actual - predicted|) / sum(|actual|) * 100
    wape = (np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))) * 100
    
    return {
        "MAE": mae, 
        "RMSE": rmse, 
        "R2": r2,
        "WAPE": wape
    }

results = []

def add_result(name, y_pred):
    m = evaluate(y_test, y_pred)
    results.append({"Model": name, **m})
    
print("✅ Metrics functions defined (with WAPE)")

✅ Metrics functions defined (with WAPE)


In [14]:
# =========================
# 2) Baselines（超重要）
# =========================
# 🚨 exp08変更点: 翌日予測なので、baselineは「今日の値 = 明日の予測」

# test_features.csvには call_num と target_next_day の両方が保存されている
test_full = pd.read_csv("../output/exp08/test_features.csv", parse_dates=["cdr_date"])
baseline_current_day = test_full['call_num'].values

# (A) 当日踏襲: y_hat(t+1) = call_num(t)
add_result("Baseline: current_day (今日=明日)", baseline_current_day)

# (B) 先週同曜日踏襲: yhat = lag_7
if "lag_7" in X_test.columns:
    add_result("Baseline: lag_7 (先週同曜日)", X_test["lag_7"].values)
else:
    print("[skip] lag_7 not found")

# (C) 過去7日平均: yhat = ma_7
if "ma_7" in X_test.columns:
    add_result("Baseline: ma_7 (過去7日平均)", X_test["ma_7"].values)
else:
    print("[skip] ma_7 not found")

# (D) 混合（地味に強いことが多い）
if "lag_7" in X_test.columns:
    yhat = 0.7 * baseline_current_day + 0.3 * X_test["lag_7"].values
    add_result("Baseline: 0.7*current + 0.3*lag_7", yhat)

In [15]:
# # =========================
# # 3) Models（まずはsklearnで堅く）
# # =========================
# models = {}

# # Ridge（説明しやすく、外れに強め。まず置く）
# models["Ridge"] = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("scaler", StandardScaler()),
#     ("model", Ridge(alpha=1.0, random_state=42))
# ])

# # ランダムフォレスト（非線形・相互作用）
# models["RandomForest"] = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("model", RandomForestRegressor(
#         n_estimators=500,
#         max_depth=None,
#         min_samples_split=5,
#         min_samples_leaf=2,
#         random_state=42,
#         n_jobs=-1
#     ))
# ])

# # ExtraTrees（RFより強いことも多い）
# models["ExtraTrees"] = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("model", ExtraTreesRegressor(
#         n_estimators=800,
#         max_depth=None,
#         min_samples_split=5,
#         min_samples_leaf=2,
#         random_state=42,
#         n_jobs=-1
#     ))
# ])

# # 勾配ブースティング（古典）
# models["GradientBoosting"] = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("model", GradientBoostingRegressor(random_state=42))
# ])

# # HistGradientBoosting（高速で強いこと多い）
# models["HistGradientBoosting"] = Pipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("model", HistGradientBoostingRegressor(
#         max_depth=None,
#         learning_rate=0.05,
#         max_iter=2000,
#         random_state=42
#     ))
# ])

# for name, model in models.items():
#     print(f"Training {name}...")
#     model.fit(X_train, y_train)
#     pred = model.predict(X_test)
#     add_result(name, pred)

In [16]:
# =========================
# 4) Optional: XGBoost / LightGBM / CatBoost（入ってたら試す）
# =========================
# 時系列なので「trainの最後の一部」をvalidにしてearly stoppingする

# train_features.csvから直接読み込んで分割
train_full_for_split = pd.read_csv("../output/exp08/train_features.csv", parse_dates=["cdr_date"])

def time_valid_split_from_df(df, valid_days=60):
    """
    Holdout Validation: 訓練データを時系列でTrain/Valに分割
    """
    TARGET = "target_next_day"
    DROP_COLS = ["cdr_date", "call_num", TARGET]
    
    cutoff = df['cdr_date'].max() - pd.Timedelta(days=valid_days)
    
    tr_df = df[df['cdr_date'] < cutoff].copy()
    va_df = df[df['cdr_date'] >= cutoff].copy()
    
    X_tr = tr_df.drop(columns=DROP_COLS)
    y_tr = tr_df[TARGET].astype(float)
    X_va = va_df.drop(columns=DROP_COLS)
    y_va = va_df[TARGET].astype(float)
    
    return X_tr, y_tr, X_va, y_va

X_tr, y_tr, X_va, y_va = time_valid_split_from_df(train_full_for_split, valid_days=60)

print(f"Training set: {X_tr.shape}, Validation set: {X_va.shape}")

Training set: (348, 35), Validation set: (43, 35)


In [17]:
# XGBoost
try:
    from xgboost import XGBRegressor

    xgb = XGBRegressor(
        n_estimators=5000,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1
    )
    print("Training XGBoost...")
    xgb.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        verbose=False
    )
    pred = xgb.predict(X_test)
    add_result("XGBoost", pred)
except Exception as e:
    print("[skip] XGBoost:", e)

Training XGBoost...


In [18]:
# LightGBM
try:
    import lightgbm as lgb

    lgbm = lgb.LGBMRegressor(
        n_estimators=20000,
        learning_rate=0.02,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    print("Training LightGBM...")
    lgbm.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="l1",
        callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)]
    )
    pred = lgbm.predict(X_test)
    add_result("LightGBM", pred)
except Exception as e:
    print("[skip] LightGBM:", e)

Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015175 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2196
[LightGBM] [Info] Number of data points in the train set: 348, number of used features: 35
[LightGBM] [Info] Start training from score 136.114943
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

In [19]:
# CatBoost
try:
    from catboost import CatBoostRegressor

    cat = CatBoostRegressor(
        iterations=20000,
        learning_rate=0.03,
        depth=8,
        loss_function="MAE",
        random_seed=42,
        verbose=False
    )
    print("Training CatBoost...")
    cat.fit(X_tr, y_tr, eval_set=(X_va, y_va), use_best_model=True)
    pred = cat.predict(X_test)
    add_result("CatBoost", pred)
except Exception as e:
    print("[skip] CatBoost:", e)

Training CatBoost...


In [20]:
# =========================
# 5) Show results（WAPE対応版）
# =========================
res_df = pd.DataFrame(results).sort_values("MAE").reset_index(drop=True)
print("\n" + "="*80)
print("=== Results (sorted by MAE) ===")
print("="*80)
print(res_df.to_string(index=False))
print("="*80)

# WAPEでもソート表示
print("\n" + "="*80)
print("=== Results (sorted by WAPE) ===")
print("="*80)
res_df_wape = res_df.sort_values("WAPE").reset_index(drop=True)
print(res_df_wape.to_string(index=False))
print("="*80)

# 保存
import os
output_dir = '../output/exp08'
os.makedirs(output_dir, exist_ok=True)
res_df.to_csv(f"{output_dir}/model_results.csv", index=False)
print(f"\n✅ Saved: {output_dir}/model_results.csv")


=== Results (sorted by MAE) ===
                            Model       MAE      RMSE        R2      WAPE
                         CatBoost 31.593947 39.773787  0.606597 32.319901
                         LightGBM 45.754600 53.691686  0.283100 46.805933
                          XGBoost 47.608386 59.560069  0.117825 48.702315
Baseline: 0.7*current + 0.3*lag_7 48.972308 68.757881 -0.175680 50.097576
    Baseline: current_day (今日=明日) 50.415385 74.360763 -0.375092 51.573812
          Baseline: ma_7 (過去7日平均) 57.749451 67.932329 -0.147618 59.076397
          Baseline: lag_7 (先週同曜日) 67.861538 86.400053 -0.856403 69.420837

=== Results (sorted by WAPE) ===
                            Model       MAE      RMSE        R2      WAPE
                         CatBoost 31.593947 39.773787  0.606597 32.319901
                         LightGBM 45.754600 53.691686  0.283100 46.805933
                          XGBoost 47.608386 59.560069  0.117825 48.702315
Baseline: 0.7*current + 0.3*lag_7 48.972308 6

In [21]:
# =========================
# 6) Visualization
# =========================
# ベストモデルの予測を取得
try:
    best_pred = cat.predict(X_test)
    model_name = "CatBoost"
except:
    try:
        best_pred = lgbm.predict(X_test)
        model_name = "LightGBM"
    except:
        try:
            best_pred = xgb.predict(X_test)
            model_name = "XGBoost"
        except:
            best_pred = models["Ridge"].predict(X_test)
            model_name = "Ridge"

mae = mean_absolute_error(y_test, best_pred)
rmse = mean_squared_error(y_test, best_pred, squared=False)
r2 = r2_score(y_test, best_pred)

print(f"\nBest Model: {model_name}")
print(f"Test MAE : {mae:.3f}")
print(f"Test RMSE: {rmse:.3f}")
print(f"Test R2  : {r2:.3f}")

# 🚨 修正: test_meta ではなく test_full を使う
plot_df = pd.DataFrame({
    "date": test_full['cdr_date'].values,  # ← ここを修正
    "actual": y_test.values,
    "pred": np.asarray(best_pred, dtype=float)
}).sort_values("date")

# 図1：時系列
plt.figure(figsize=(12, 4))
plt.plot(plot_df["date"], plot_df["actual"], label="Actual (Next Day)", marker='o')
plt.plot(plot_df["date"], plot_df["pred"], label="Predicted", marker='x')
plt.title(f"exp03: Next-Day Prediction | {model_name} | MAE={mae:.1f}, RMSE={rmse:.1f}, R2={r2:.2f}")
plt.xlabel("Date")
plt.ylabel("Calls (Next Day)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f"{output_dir}/prediction_timeseries.png", dpi=150)
plt.show()

# 図2：散布図
plt.figure(figsize=(5.5, 5.5))
plt.scatter(plot_df["actual"], plot_df["pred"], alpha=0.6)
mn = min(plot_df["actual"].min(), plot_df["pred"].min())
mx = max(plot_df["actual"].max(), plot_df["pred"].max())
plt.plot([mn, mx], [mn, mx], 'r--')
plt.title("Predicted vs Actual (Next Day)")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.tight_layout()
plt.savefig(f"{output_dir}/prediction_scatter.png", dpi=150)
plt.show()

print(f"\n✅ Plots saved to {output_dir}/")
print("="*60)
print("exp03 complete!")
print("="*60)


Best Model: CatBoost
Test MAE : 31.594
Test RMSE: 39.774
Test R2  : 0.607


NameError: name 'plt' is not defined

## ✅ exp03完了

### exp01との違い
- **目的変数**: `call_num` → `call_num.shift(-1)` （翌日予測）
- **ベースライン**: `lag_1` → `call_num`（当日の値）
- **特徴量**: 完全に同じ
- **モデル**: 完全に同じ

### 出力ファイル
- `output/exp03/train_features.csv` - 訓練データ（target_next_day付き）
- `output/exp03/test_features.csv` - テストデータ（target_next_day付き）
- `output/exp03/feature_correlations.csv` - 特徴量相関
- `output/exp03/model_results.csv` - 全モデルの評価結果
- `output/exp03/prediction_timeseries.png` - 時系列予測プロット
- `output/exp03/prediction_scatter.png` - 散布図

In [ ]:
# exp08の特徴量重要度を取得・可視化
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CatBoostモデルから特徴量重要度を取得
# （既にモデル学習セクションを実行済みの前提）

print("="*60)
print("exp08 - CatBoost Feature Importance")
print("="*60)

# CatBoostモデルが学習済みであることを確認
try:
    # 特徴量重要度を取得
    feature_importance = cat.feature_importances_
    feature_names = X_test.columns.tolist()
    
    # DataFrameに整形
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False).reset_index(drop=True)
    
    print(f"\nTotal features: {len(importance_df)}")
    print(f"\nTop 20 features:")
    print(importance_df.head(20).to_string(index=False))
    
    # 保存
    import os
    output_dir = '../output/exp08'
    os.makedirs(output_dir, exist_ok=True)
    importance_df.to_csv(f'{output_dir}/feature_importance.csv', index=False)
    print(f"\n✅ Saved to {output_dir}/feature_importance.csv")
    
    # 可視化1: Top 20
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 横棒グラフ
    ax = axes[0]
    top20 = importance_df.head(20).sort_values('importance')
    ax.barh(range(len(top20)), top20['importance'])
    ax.set_yticks(range(len(top20)))
    ax.set_yticklabels(top20['feature'])
    ax.set_xlabel('Importance')
    ax.set_title(f'exp08 - Top 20 Features (Total: {len(importance_df)})')
    ax.grid(True, alpha=0.3)
    
    # 棒グラフ（全特徴量）
    ax = axes[1]
    ax.bar(range(len(importance_df)), importance_df['importance'])
    ax.set_xlabel('Feature Index (sorted by importance)')
    ax.set_ylabel('Importance')
    ax.set_title('exp08 - All Features Importance Distribution')
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/feature_importance_plot.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # 可視化2: カテゴリ別の重要度
    print("\n" + "="*60)
    print("Feature Importance by Category")
    print("="*60)
    
    # カテゴリ分類
    categories = {
        'Lag': [f'lag_{i}' for i in [1,2,3,5,7,14,30]],
        'Moving Avg': ['ma_3', 'ma_7', 'ma_14', 'ma_30'],
        'Moving Std': ['ma_std_3', 'ma_std_7', 'ma_std_14', 'ma_std_30'],
        'Time': ['dow', 'day_of_month', 'month', 'quarter', 'year', 
                 'days_from_start', 'day_of_year', 'week_of_year',
                 'is_month_start', 'is_month_end', 'woy', 'wom'],
        'External': ['cm_flg', 'acc_get_cnt', 'search_cnt'],
        'Aggregated': ['cm_7d', 'gt_ma_7', 'acc_ma_7', 'dow_avg'],
        'Other': ['day_before_holiday_flag']
    }
    
    category_importance = {}
    for cat_name, features in categories.items():
        cat_imp = importance_df[importance_df['feature'].isin(features)]['importance'].sum()
        category_importance[cat_name] = cat_imp
    
    cat_df = pd.DataFrame({
        'Category': category_importance.keys(),
        'Total Importance': category_importance.values()
    }).sort_values('Total Importance', ascending=False)
    
    print(cat_df.to_string(index=False))
    
    # カテゴリ別の可視化
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(cat_df['Category'], cat_df['Total Importance'])
    ax.set_ylabel('Total Importance')
    ax.set_title('exp08 - Feature Importance by Category')
    ax.grid(True, alpha=0.3, axis='y')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f'{output_dir}/feature_importance_by_category.png', dpi=150, bbox_inches='tight')
    plt.show()
    
except NameError:
    print("⚠️ CatBoostモデル（cat）がまだ学習されていません")
    print("モデル学習セクションを先に実行してください")
except Exception as e:
    print(f"エラー: {e}")


In [ ]:
# =========================
# WAPE詳細分析
# =========================
import matplotlib.pyplot as plt
import numpy as np

print("\n" + "="*80)
print("WAPE Analysis")
print("="*80)

# ベストモデル（MAE基準）
best_mae_model = res_df.iloc[0]['Model']
best_mae_wape = res_df.iloc[0]['WAPE']

# ベストモデル（WAPE基準）
best_wape_model = res_df_wape.iloc[0]['Model']
best_wape_value = res_df_wape.iloc[0]['WAPE']

print(f"\nBest model by MAE:  {best_mae_model} (WAPE: {best_mae_wape:.2f}%)")
print(f"Best model by WAPE: {best_wape_model} (WAPE: {best_wape_value:.2f}%)")

# WAPEの解釈
print(f"\nWAPE Interpretation:")
print(f"  WAPE < 10%:  Excellent")
print(f"  WAPE < 20%:  Good")
print(f"  WAPE < 30%:  Fair")
print(f"  WAPE >= 30%: Needs improvement")

if best_wape_value < 10:
    grade = "Excellent ⭐⭐⭐"
elif best_wape_value < 20:
    grade = "Good ⭐⭐"
elif best_wape_value < 30:
    grade = "Fair ⭐"
else:
    grade = "Needs improvement"

print(f"\nBest model WAPE: {best_wape_value:.2f}% → {grade}")

# 可視化: MAE vs WAPE
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MAE比較
ax = axes[0]
models = res_df['Model'].values
mae_values = res_df['MAE'].values
colors = ['green' if i == 0 else 'skyblue' for i in range(len(models))]
bars = ax.barh(range(len(models)), mae_values, color=colors)
ax.set_yticks(range(len(models)))
ax.set_yticklabels(models, fontsize=8)
ax.set_xlabel('MAE')
ax.set_title('Model Comparison: MAE')
ax.grid(True, alpha=0.3, axis='x')
# 値ラベル
for i, (bar, val) in enumerate(zip(bars, mae_values)):
    ax.text(val, i, f' {val:.1f}', va='center', fontsize=8)

# WAPE比較
ax = axes[1]
wape_values = res_df_wape['WAPE'].values
models_wape = res_df_wape['Model'].values
colors = ['green' if i == 0 else 'lightcoral' for i in range(len(models_wape))]
bars = ax.barh(range(len(models_wape)), wape_values, color=colors)
ax.set_yticks(range(len(models_wape)))
ax.set_yticklabels(models_wape, fontsize=8)
ax.set_xlabel('WAPE (%)')
ax.set_title('Model Comparison: WAPE')
ax.grid(True, alpha=0.3, axis='x')
# 値ラベル
for i, (bar, val) in enumerate(zip(bars, wape_values)):
    ax.text(val, i, f' {val:.1f}%', va='center', fontsize=8)

plt.tight_layout()
plt.savefig(f"{output_dir}/metrics_comparison_with_wape.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Plot saved: {output_dir}/metrics_comparison_with_wape.png")